In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from openpyxl import load_workbook

In [2]:
# Scrape IMDB for list of movie titles
def getTitles(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    header = soup.find_all('h3', class_='lister-item-header')

    titles = []
    for item in header:
        titles.append(item.find('a').get_text())
    return titles

In [3]:
# Find ZH wiki page link to each movie title
def getLinks(titles):
    links = []
    for idx, title in enumerate(titles):
        title = title.replace(' ','_')
        print('Now processing title number' + str(idx))
        try:
            try:
                tmp = []
                path = 'https://en.wikipedia.org/wiki/' + title + '_(film)'
                page = requests.get(path)
                soup = BeautifulSoup(page.content, 'html.parser')
                li = soup.find('li', class_='interlanguage-link interwiki-zh')
                lin = li.find('a')
                link = lin['href']
                links.append(link)
            except:
                tmp = []
                path = 'https://en.wikipedia.org/wiki/' + title
                page = requests.get(path)
                soup = BeautifulSoup(page.content, 'html.parser')
                li = soup.find('li', class_='interlanguage-link interwiki-zh')
                lin = li.find('a')
                link = lin['href']
                links.append(link)
        except:
            links.append('404')
            print('Not Found.')
    return links

In [4]:
# Get 3 translated titles of China, Hong Kong and Taiwan:
def getTranslation(links):
    titlesCH = []
    titlesHK = []
    titlesTW = []
    for link in links:
        try:
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')
            table = soup.find('table', class_='infobox vevent')
            td = table.find_all('td')[-3:]
            titlesCH.append(td[0].get_text())
            titlesHK.append(td[1].get_text())
            titlesTW.append(td[2].get_text())
        except:
            titlesCH.append('null')
            titlesHK.append('null')
            titlesTW.append('null')
    return titlesCH, titlesHK, titlesTW

In [5]:
imdbPath = 'https://www.imdb.com/search/title?year=2016&title_type=feature&sort=num_votes,desc&page=4&ref_=adv_nxt'
titles = getTitles(imdbPath)

In [6]:
links = getLinks(titles)

Now processing title number0
Now processing title number1
Not Found.
Now processing title number2
Not Found.
Now processing title number3
Not Found.
Now processing title number4
Not Found.
Now processing title number5
Not Found.
Now processing title number6
Not Found.
Now processing title number7
Not Found.
Now processing title number8
Now processing title number9
Now processing title number10
Now processing title number11
Not Found.
Now processing title number12
Not Found.
Now processing title number13
Now processing title number14
Not Found.
Now processing title number15
Now processing title number16
Not Found.
Now processing title number17
Not Found.
Now processing title number18
Now processing title number19
Now processing title number20
Not Found.
Now processing title number21
Now processing title number22
Not Found.
Now processing title number23
Now processing title number24
Not Found.
Now processing title number25
Not Found.
Now processing title number26
Not Found.
Now processin

In [7]:
titlesCH, titlesHK, titlesTW = getTranslation(links)

In [8]:
results = pd.DataFrame({
    'TitlesEN': titles,
    'TitlesCH': titlesCH,
    'TitlesHK': titlesHK,
    'TitlesTW': titlesTW,
    'wikiZH': links
})

# Write to an existing excel file and create new sheet
with pd.ExcelWriter('final.xlsx', engine='openpyxl') as writer:
    writer.book = load_workbook('final.xlsx')
    results.to_excel(writer, sheet_name='2016_p4')

/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
